# Carregando o dataframe

In [2]:
import os
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# from sklearn.svm import SVR
# from sklearn import svm

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from math import sqrt



In [3]:
base_folder = os.getcwd()
for dirname, _, filenames in os.walk(os.path.join(base_folder, 'input')):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/lucashelal/Workspace/retail_case/input/comp_prices.csv
/Users/lucashelal/Workspace/retail_case/input/sales.csv


In [4]:
df_comp_price = pd.read_csv(os.path.join(base_folder,'input/comp_prices.csv'))


## Contéudo do Dataframe


comp_prices.csv

> PROD_ID:  ID do produto. Contém 9 diferentes produtos (P1 - P9).

> DATE_EXTRACTION: Data e Hora da extração dos dados dos concorrentes. Formato: YYYY-MM-DD HH:MM:SS

> COMPETITOR: ID dos concorrentes. Contém6 diferentes produtos (C1 - C6).

> COMPETITOR_PRICE: Preços dos Concorrentes por produto, no qual pode depender do meio de pagamento (PAY_TYPE).

> PAY_TYPE:  Meio de Pagamento (1- Pagamento diferido, 2- Pagamento imediato)

In [5]:
df_comp_price.head()

,PROD_ID,DATE_EXTRACTION,COMPETITOR,COMPETITOR_PRICE,PAY_TYPE
0,P6,2015-01-30 08:11:38,C3,1898.00,1
1,P6,2015-01-30 08:11:38,C1,1898.00,1
2,P6,2015-01-30 08:11:38,C6,1999.90,1
3,P6,2015-01-31 20:10:14,C2,1894.88,2
4,P6,2015-01-31 20:10:14,C3,1894.88,2


# Preparando dataframe para o modelo

## Remover possíveis outliers por produto

In [8]:
def remove_outliers_sales(df, col_id, col_outlier):
    column_ID = df[col_id].unique()
    for i in (column_ID):
        df_loc_col = df.loc[df[col_id] == i][col_outlier]
        Q1 = df_loc_col.quantile(0.05)
        Q3 = df_loc_col.quantile(0.95)
        IQR = Q3 - Q1
        
        _filter = (df_loc_col >= Q1 - 1.5*IQR) & (df_loc_col <= Q3 + 1.5*IQR)

        df.loc[df[col_id] == i, [col_outlier]] = df_loc_col.loc[_filter]

In [9]:
df = df_comp_price.copy()
remove_outliers_sales(df, col_id = 'PROD_ID', col_outlier = 'COMPETITOR_PRICE')

### Removendo do dataframe as colunas que ficaram com null

In [10]:
df = df.dropna()
df = df.reset_index(drop=True)

In [11]:
outliers = df_comp_price['COMPETITOR_PRICE'].count() - df['COMPETITOR_PRICE'].count()
print('Foram removidos {} outliers'.format(outliers))

Foram removidos 82 outliers


### Adicionando colunas mês e dia e hora (MONTH / DAY / HOUR)

- A intenção aqui é melhorar e aumentar nossos dados com o que já temos.

In [14]:
df['DATE_EXTRACTION'] =  pd.to_datetime(df['DATE_EXTRACTION'], format='%Y-%m-%d %H:%M:%S')
df['MONTH'] = df['DATE_EXTRACTION'].dt.month
df['DAY'] = df['DATE_EXTRACTION'].dt.weekday
df['HOUR'] = df['DATE_EXTRACTION'].dt.hour
df.head()

,PROD_ID,DATE_EXTRACTION,COMPETITOR,COMPETITOR_PRICE,PAY_TYPE,MONTH,DAY,HOUR
0,P6,2015-01-30 08:11:38,C3,1898.00,1,1,4,8
1,P6,2015-01-30 08:11:38,C1,1898.00,1,1,4,8
2,P6,2015-01-30 08:11:38,C6,1999.90,1,1,4,8
3,P6,2015-01-31 20:10:14,C2,1894.88,2,1,5,20
4,P6,2015-01-31 20:10:14,C3,1894.88,2,1,5,20


### NORMALIZAR

- Quando falamos em normalizar os dados, no nosso caso deixar entre 0-1 é para que facilite para a máquina "entender" sobre os valores. Nesse cenário normalizando "todas" as colunas ficar mais "fácil" para a máquina aprender o padrão.

- Porém aqui resolvi apenas normalizar o preço de cada concorrente, somente para vizualizarmos esse cenário.

In [16]:
def normalize_col(df, col_name):
    
    norm = df[[col_name]].values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    
    x_scaled = min_max_scaler.fit_transform(norm)
    new_col_name = col_name + '_NORMALIZED'
    
    df_normalized = pd.DataFrame(x_scaled)
    df[[new_col_name]] = df_normalized


In [17]:
normalize_col(df, 'COMPETITOR_PRICE')

In [20]:
df.head()

,PROD_ID,DATE_EXTRACTION,COMPETITOR,COMPETITOR_PRICE,PAY_TYPE,MONTH,DAY,HOUR,COMPETITOR_PRICE_NORMALIZED
0,P6,2015-01-30 08:11:38,C3,1898.00,1,1,4,8,0.630490
1,P6,2015-01-30 08:11:38,C1,1898.00,1,1,4,8,0.630490
2,P6,2015-01-30 08:11:38,C6,1999.90,1,1,4,8,0.672239
3,P6,2015-01-31 20:10:14,C2,1894.88,2,1,5,20,0.629212
4,P6,2015-01-31 20:10:14,C3,1894.88,2,1,5,20,0.629212


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50032 entries, 0 to 50031
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   PROD_ID                      50032 non-null  object        
 1   DATE_EXTRACTION              50032 non-null  datetime64[ns]
 2   COMPETITOR                   50032 non-null  object        
 3   COMPETITOR_PRICE             50032 non-null  float64       
 4   PAY_TYPE                     50032 non-null  int64         
 5   MONTH                        50032 non-null  int64         
 6   DAY                          50032 non-null  int64         
 7   HOUR                         50032 non-null  int64         
 8   COMPETITOR_PRICE_NORMALIZED  50032 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 3.4+ MB


### Adicionando a coluna quantidade

- A ideia aqui é, novamente, incrementar nossos dados com o que já temos.
- Resolvi agrupar nossos dados e ver quais linhas eram iguais, no caso, podemos pensar que caso tenhamos três linhas iguais, podemos concluir que naquele momento foram feitos 3 pedidos, sendo mais específico, naquela hora.

**Observação**

- Já deixando como ideia para trabalhos futuros: a criação de outras colunas, como quantidade vendida no dia, semana e mês.

In [22]:
df_new = df[['PROD_ID', 'COMPETITOR', 'PAY_TYPE', 'MONTH', 'DAY', 'HOUR', 'COMPETITOR_PRICE_NORMALIZED', 'COMPETITOR_PRICE']]

#### Agrupando itens e caso sejam igual, são agrupados e será considerado uma única compra com quantidade.

In [23]:
df_new = df_new.groupby(df_new.columns.tolist(),as_index=False).size()

In [24]:
df_new = df_new.rename(columns={"size": "QUANTITY"})

In [25]:
df_new.head()

,PROD_ID,COMPETITOR,PAY_TYPE,MONTH,DAY,HOUR,COMPETITOR_PRICE_NORMALIZED,COMPETITOR_PRICE,QUANTITY
0,P1,C1,1,3,0,20,0.467019,1499.0,1
1,P1,C1,1,3,1,9,0.411095,1362.5,1
2,P1,C1,1,3,1,20,0.467019,1499.0,1
3,P1,C1,1,3,2,8,0.467019,1499.0,1
4,P1,C1,1,3,2,20,0.411095,1362.5,1


### Dataframe para o Modelo

In [27]:
df_model = df_new.copy()

#### Discretizando a coluna 'COMPETITOR'

In [28]:
df_model = pd.get_dummies(df_model, columns=['COMPETITOR'])

In [29]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33106 entries, 0 to 33105
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   PROD_ID                      33106 non-null  object 
 1   PAY_TYPE                     33106 non-null  int64  
 2   MONTH                        33106 non-null  int64  
 3   DAY                          33106 non-null  int64  
 4   HOUR                         33106 non-null  int64  
 5   COMPETITOR_PRICE_NORMALIZED  33106 non-null  float64
 6   COMPETITOR_PRICE             33106 non-null  float64
 7   QUANTITY                     33106 non-null  int64  
 8   COMPETITOR_C1                33106 non-null  uint8  
 9   COMPETITOR_C2                33106 non-null  uint8  
 10  COMPETITOR_C3                33106 non-null  uint8  
 11  COMPETITOR_C4                33106 non-null  uint8  
 12  COMPETITOR_C5                33106 non-null  uint8  
 13  COMPETITOR_C6   

### Separando os dados em entrada (input) e alvo (target/label), respectivamente X, y.

In [31]:
X, y = df_model[['PROD_ID', 'MONTH', 'DAY', 'HOUR', 'COMPETITOR_C1', 'COMPETITOR_C2', 'COMPETITOR_C3', 'COMPETITOR_C4', 'COMPETITOR_C5', 'COMPETITOR_C6', 'QUANTITY']], df_model['COMPETITOR_PRICE_NORMALIZED']

#### Mapeando o PROD_ID em inteiros

In [33]:
PROD_ID = {'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5, 'P6': 6, 'P7': 7, 'P8': 8, 'P9': 9} 
X = X.replace(PROD_ID)
X

,PROD_ID,MONTH,DAY,HOUR,COMPETITOR_C1,COMPETITOR_C2,COMPETITOR_C3,COMPETITOR_C4,COMPETITOR_C5,COMPETITOR_C6,QUANTITY
0,1,3,0,20,1,0,0,0,0,0,1
1,1,3,1,9,1,0,0,0,0,0,1
2,1,3,1,20,1,0,0,0,0,0,1
3,1,3,2,8,1,0,0,0,0,0,1
4,1,3,2,20,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
33101,9,10,5,20,0,0,0,0,0,1,2
33102,9,10,5,20,0,0,0,0,0,1,2
33103,9,10,6,8,0,0,0,0,0,1,2
33104,9,10,6,20,0,0,0,0,0,1,2


### Dividindo os dados em Treino (80%) em Teste (20%)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Aqui poderiamos testar diversos algoritmos/modelos, inicialmente testei com o SVM e com os seus diversos kernels, mas estava demorando para rodar na minha máquina, resolvi  utilizar o KNN com as configurações Default que se teve bons resultados.

Como gosto de dizer:  "a imaginação é o limite, rs.."

# Aprendizagem

## Treino e Teste - KNN

In [35]:
Knn = neighbors.KNeighborsRegressor()
Knn.fit(X_train, y_train)  
error = sqrt(metrics.mean_squared_error( y_test, Knn.predict(X_test))) 
print('RMSE:', error)

RMSE: 0.06155424613100982


In [36]:
Knn.score(X_train, y_train) 


0.9421724049967611

94,21% de acc

In [37]:
# Caso queira prever alguns resultados, dado determinada de entrada

# result = Knn.predict(X_test)
# print(result)

## Modelo II: Discretizando também o 'PROD_ID'

In [39]:
df_model_two = pd.get_dummies(df_model)

In [41]:
# df_model_two

In [42]:
X, y = df_model_two[['PROD_ID_P1', 'PROD_ID_P2', 'PROD_ID_P3', 'PROD_ID_P4', 'PROD_ID_P5', 'PROD_ID_P6', 'PROD_ID_P7', 'PROD_ID_P8', 'PROD_ID_P9',  'MONTH', 'DAY', 'HOUR', 'COMPETITOR_C1', 'COMPETITOR_C2', 'COMPETITOR_C3', 'COMPETITOR_C4', 'COMPETITOR_C5', 'COMPETITOR_C6', 'QUANTITY']], df_model['COMPETITOR_PRICE_NORMALIZED']

In [43]:
X

,PROD_ID_P1,PROD_ID_P2,PROD_ID_P3,PROD_ID_P4,PROD_ID_P5,PROD_ID_P6,PROD_ID_P7,PROD_ID_P8,PROD_ID_P9,MONTH,DAY,HOUR,COMPETITOR_C1,COMPETITOR_C2,COMPETITOR_C3,COMPETITOR_C4,COMPETITOR_C5,COMPETITOR_C6,QUANTITY
0,1,0,0,0,0,0,0,0,0,3,0,20,1,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,3,1,9,1,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,3,1,20,1,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,3,2,8,1,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,3,2,20,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33101,0,0,0,0,0,0,0,0,1,10,5,20,0,0,0,0,0,1,2
33102,0,0,0,0,0,0,0,0,1,10,5,20,0,0,0,0,0,1,2
33103,0,0,0,0,0,0,0,0,1,10,6,8,0,0,0,0,0,1,2
33104,0,0,0,0,0,0,0,0,1,10,6,20,0,0,0,0,0,1,2


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [86]:
Knn = neighbors.KNeighborsRegressor()
Knn.fit(X_train, y_train)  
error = sqrt(metrics.mean_squared_error( y_test, Knn.predict(X_test))) 
print('RMSE:', error)

RMSE: 0.04168975667188638


In [84]:
Knn.score(X_train, y_train)  # R2 of the KNN model


0.9719549579704859

97,20% de acc

Se teve um ganho de 3% com a discretização das variável PROD_ID

Logo, nosso modelo de previsão de preços baseado na base de dados dos correntes tem uma acertividade de 97%